# Jacobian testing before going for dynamics

In [1]:
import pybullet as p
import pybullet_data
import numpy as np
import math
import time
from Dynamics_full.config import robot_config
from Dynamics_full.src.kinematics.kinematic_calculations import compute_geometric_jacobian
import MathUtils

In [2]:
URDF_PATH = r"C:\dev\control-sw-tools\Dynamics_full\ArmModels\urdfs\P4\P4_Contra-Angle_right.urdf"

In [3]:
def get_link_index_by_name_pb_jac(robot_id, target_link_name): # Copied from your 01Test
    base_name = p.getBodyInfo(robot_id)[0].decode('UTF-8')
    if target_link_name == base_name or target_link_name == "Base": return -1
    for i in range(p.getNumJoints(robot_id)):
        info = p.getJointInfo(robot_id, i)
        link_name = info[12].decode('UTF-8')
        if link_name == target_link_name: return i
    print(f"Warning: Link '{target_link_name}' not found.")
    return None

def get_joint_indices_by_names_pb_jac(robot_id, target_joint_names): # Modified
    joint_indices = []
    pb_joint_info = {p.getJointInfo(robot_id, i)[1].decode('UTF-8'): i for i in range(p.getNumJoints(robot_id))}
    for name in target_joint_names:
        if name in pb_joint_info:
            joint_indices.append(pb_joint_info[name])
        else:
            print(f"Warning: Joint '{name}' not found in PyBullet model.")
            return None
    return joint_indices

In [4]:
def main_jacobian_test():
    physicsClientId = p.connect(p.GUI) # Or GUI if you want to see it
    p.setAdditionalSearchPath(pybullet_data.getDataPath())
    planeId = p.loadURDF("plane.urdf")
    p.changeVisualShape(planeId, -1, rgbaColor=[0.8, 0.8, 0.8, 0.3])

    robot_id = p.loadURDF(URDF_PATH, useFixedBase=True)

    test_joint_angles = [0.0, 1.54, 0.0, 0.0, 0.0, 0.0, 0.0]
    # test_joint_angles = [0.1] * robot_config.NUM_JOINTS # Small non-zero

    ee_link_name_urdf = robot_config.LINK_NAMES_IN_KDL_ORDER[-1]

    pb_ee_link_index = get_link_index_by_name_pb_jac(robot_id, ee_link_name_urdf)
    if pb_ee_link_index is None:
        print(f"CRITICAL: EE Link '{ee_link_name_urdf}' not found in URDF for PyBullet Jacobian. Exiting.")
        p.disconnect()
        return

    pb_actuated_joint_indices = get_joint_indices_by_names_pb_jac(robot_id, robot_config.ACTUATED_JOINT_NAMES)
    if pb_actuated_joint_indices is None or len(pb_actuated_joint_indices) != robot_config.NUM_JOINTS:
        print("CRITICAL: Could not map all actuated joint names to PyBullet indices. Exiting.")
        p.disconnect()
        return

    for i in range(robot_config.NUM_JOINTS):
        p.resetJointState(robot_id, pb_actuated_joint_indices[i], test_joint_angles[i])

    neocis_fk_for_custom_J = True
    J_custom = compute_geometric_jacobian(
        robot_config.KDL_CHAIN,
        test_joint_angles,
        neocis_convention_fk=neocis_fk_for_custom_J
    )
    print("\n--- Custom Jacobian ---")
    print(f"Using neocis_convention={neocis_fk_for_custom_J} for FK in custom Jacobian.")
    print(np.round(J_custom, 2))

    # --- Calculate Jacobian using PyBullet ---
    # PyBullet's calculateJacobian needs:
    # - bodyUniqueId
    # - linkIndex (of the end-effector point)
    # - localPosition (of the point on the link, relative to link's CoM frame, or link's URDF origin frame - check docs)
    #   For geometric Jacobian, this is usually [0,0,0] relative to the EE link's *frame origin*.
    # - objPositions (current joint angles)
    # - objVelocities (current joint velocities - can be zero for geometric J)
    # - objAccelerations (current joint accelerations - can be zero for geometric J)

    # Get the position of the EE link's origin in its own frame (which is [0,0,0])
    # The Jacobian is calculated for a point on the link. If KDL_CHAIN[-1] defines the EE frame origin,
    # then localPositionOnLink should be [0,0,0] for that frame.
    # PyBullet's link frame origin is what's defined in the URDF for that link.
    # If your KDL's last transform points to the origin of the URDF EE link, then [0,0,0] is correct.
    link_state_ee = p.getLinkState(robot_id, pb_ee_link_index, computeForwardKinematics=1)
    # T_world_URDFLinkCoM = get_matrix_from_pose_pb((link_state_ee[0],link_state_ee[1])) # Transform to Link CoM
    # T_world_URDFLinkFrame = get_matrix_from_pose_pb((link_state_ee[4],link_state_ee[5])) # Transform to Link URDF Frame Origin

    # Let's calculate for the origin of the EE link's URDF frame
    local_point_on_ee_link = [0.0, 0.0, 0.0]

    zero_vec = [0.0] * robot_config.NUM_JOINTS
    J_pb_tuple = p.calculateJacobian(
        bodyUniqueId=robot_id,
        linkIndex=pb_ee_link_index,
        localPosition=local_point_on_ee_link,
        objPositions=test_joint_angles,
        objVelocities=zero_vec, # Not used for geometric Jacobian computation itself
        objAccelerations=zero_vec # Not used
    )
    J_pb_linear = np.array(J_pb_tuple[0])   # Linear part (dx/dq, dy/dq, dz/dq)
    J_pb_angular = np.array(J_pb_tuple[1]) # Angular part (droll/dq, dpitch/dq, dyaw/dq)
    J_pb = np.vstack((J_pb_linear, J_pb_angular))

    print("\n--- PyBullet Jacobian ---")
    print(np.round(J_pb, 2))

    # --- Compare ---
    diff_J = np.abs(J_custom - J_pb)
    print(f"\n--- Comparison (Custom - PyBullet) ---")
    print(f"Max absolute difference: {np.max(diff_J):.6f}")
    print(f"Mean absolute difference: {np.mean(diff_J):.6f}")

    if np.allclose(J_custom, J_pb, atol=1e-2): # Adjust tolerance as needed
        print("\nSUCCESS: Custom Jacobian closely matches PyBullet Jacobian!")
    else:
        print("\nWARNING: Custom Jacobian differs significantly from PyBullet Jacobian.")
        print("Differences (Custom - PyBullet):")
        print(np.round(J_custom - J_pb, 2))

    p.disconnect()

In [5]:
if __name__ == "__main__":
    main_jacobian_test()


--- Custom Jacobian ---
Using neocis_convention=True for FK in custom Jacobian.
[[-0.58 -0.58  0.02  0.   -0.23 -0.01 -0.  ]
 [ 0.39  0.02 -0.    0.02  0.01 -0.23 -0.  ]
 [ 0.    0.    0.   -0.31  0.   -0.   -0.  ]
 [ 0.    0.    0.03 -1.    0.03 -1.   -0.  ]
 [ 0.    0.    1.    0.03  1.    0.03  0.  ]
 [ 1.    1.    0.    0.    0.    0.   -1.  ]]

--- PyBullet Jacobian ---
[[-0.58 -0.58  0.12  0.   -0.13 -0.    0.  ]
 [ 0.39  0.02 -0.    0.12  0.   -0.13 -0.  ]
 [ 0.    0.    0.   -0.31 -0.    0.    0.  ]
 [-0.   -0.    0.03 -1.    0.03 -1.    0.  ]
 [ 0.    0.    1.    0.03  1.    0.03 -0.  ]
 [ 1.    1.   -0.    0.   -0.   -0.   -1.  ]]

--- Comparison (Custom - PyBullet) ---
Max absolute difference: 0.098653
Mean absolute difference: 0.009686

Differences (Custom - PyBullet):
[[ 0.   0.  -0.1 -0.  -0.1 -0.  -0. ]
 [ 0.   0.   0.  -0.1  0.  -0.1 -0. ]
 [-0.  -0.  -0.   0.   0.  -0.  -0. ]
 [ 0.   0.   0.   0.   0.  -0.  -0. ]
 [-0.  -0.   0.   0.   0.   0.   0. ]
 [ 0.   0.   0.  